In [2]:
import pymysql
import redis
import pandas as pd
from itertools import batched

In [3]:
df = pd.read_csv('amazon_kitchenware.csv')

In [4]:
# convert column names to snake_case
df.columns = (df.columns
                .str.replace('(?<=[a-z])(?=[A-Z])', '_', regex=True)
                .str.lower()
             )

df = df.rename(columns={'asin': 'product_id'})[
    ~df['price/value'].isna()
]

info_df = df[['product_id', 'title', 'brand', 'description', 'stars', 'reviews_count', 'bread_crumbs', 'url']]
price_df = df[['product_id', 'price/currency', 'price/value']]

### Writing product info into Redis

In [5]:
redis_conn = redis.Redis(host='localhost', port=6379, db=0)

for batch in batched(info_df.to_dict(orient='records'), 100):
    with redis_conn.pipeline() as pipe:
        for record in batch:
            pipe.hmset(f'product:{record["product_id"]}', record)
        pipe.execute()

/var/folders/vv/v235wtnd4kb6l7ql8sx8h4fh0000gn/T/ipykernel_21152/857722342.py:6: DeprecationWarning: Pipeline.hmset() is deprecated. Use Pipeline.hset() instead.
  pipe.hmset(f'product:{record["product_id"]}', record)


### Writing product price into Postgres

In [6]:
credentials = {
    'user': 'mysql',
    'password': 'mysql',
    'database': 'mysql',
    'host': "localhost",
    'port': 3306
}

with (
    pymysql.connect(**credentials) as conn,
    conn.cursor() as cursor
):
    CREATE_TABLE_STMT = """
        CREATE TABLE product_price (
            product_id VARCHAR(10) PRIMARY KEY,
            currency VARCHAR(3),
            value FLOAT
        );
    """
    
    cursor.execute(CREATE_TABLE_STMT)
    
    for batched_rows in batched(price_df.to_dict('records'), 10):
        insert_stmt = """
            INSERT INTO product_price (product_id, currency, value)
            VALUES (%s, %s, %s)
        """
        cursor.executemany(insert_stmt, [(row['product_id'], row['price/currency'], row['price/value']) for row in batched_rows])
    conn.commit()